# Rocket motor model
Это симуляция твердотопливного ракетного двигателя.
Она сделана по образу программы [Rocki-motors](http://kia-soft.narod.ru/soft/rpro/rms/rms.rar).
Дополнительно можно рассчитать полёт ракеты.

Чтобы запустить программу, задайте настройки во 1-й ячейке и запустите её кнопкой в верхнем левом углу. Потом запустите ячейку с кодом программы (последнюю). Снизу будет вывод программы.
Чтобы сменить настройки, нужно снова запустить ячейку настроек.

Если вы просматриваете этот файл на GitHub или NotABug, перейдите сначала [сюда](https://colab.research.google.com/github/busyBeaver1/Rocket_motor_model/blob/main/Rocket_motor_model.ipynb), чтобы запустить програму.

!Эта программа НЕ расчитывает переходные состояния двигателя (выход на режим и сброс давления)! Однако переходные состояния обычно слабо влияют на общую картину и для большинства ракетомоделистов в большинстве случаев не важны.

In [ ]:
# settings
# настройки

# параметры двигателя ==================

# двигатель состоит из цилиндрических шашек с каналом в середине. В конце, за шашками также может быть замедлитель (трассер). В трассере канала нет. Между шашками должен быть проложен мгновенный воспламеняющий состав.
# про вторичный (мгновенный) воспламеняющий состав см. здесь: https://youtu.be/mQn43RASYNM

# если разделения топлива на шашки нет, установите значение ignited_faces = 0. Тогда все шашки двигателя будут работать как будто это одна большая.
# если двигатель торцевой, установите значения hole = 0. и ignited_faces = 1

# если вы хотите использовать не круглый канал в топливе, вы можете загрузить своё изображение канала и указать файл с помощью параметра custom_grain. Тогда параметр hole не используется.
# изображение должно быть квадратным. Из него будет вырезан вписанный круг (т.к. шашки круглые). Чёрным цветом на изображении обозначается топливо, белим - пустота (канал).
# если вы запускаете программу на сайте Google Colab, загрузите изображение по кнопке со значком папки слева, если локально - положите изображение в папку программы
# также желательно увеличить параметр dots до значения около 80 для большей точности
custom_grain = ... # название файла с кастомной формой канала (название должно быть в кавычках, пример: custom_grain = 'test.png'), ... - использовать круглый канал с диаметром hole
min_pressure = 0.5 # минимальное давление (атм, относительно атмосферного), при котором симуляция заканчивается. Используется только для кастомного канала. Это костыль чтобы уравнения не ломались при сильно малых Kn.
use_min_pressure = True # использовать ли min_pressure (если custom_grain = ... , то min_pressure в любом случае НЕ используется)

ignited_faces = 2 # сколько из торцов шашек зажигаются, 0, 1 или 2 (обычно 2 для шашек и 0 для двигателей, где топливо напрямую закладывается в корпус движка)
d = 28. # диаметр шашки, мм
hole = 8. # диаметр канала, мм
l = 45. # длина шашки, мм
h = 2 # количество шашек
d_t = 28. # диаметр трассера (замедлителя), мм
l_t = 0. # длина трассера, мм. Если трассер не нужен, поставьте 0.
nozzle_throat = 6. # диаметр критики сопла (диаметр самого узкого места), мм
nozzle_out = 15. # диаметр выхода из сопла, мм
d_throat = 0.2 # разгар критики сопла (увеличение диаметра), мм
d_out = 0. # разгар выхода сопла, мм. Должен быть таким чтобы выход сопла НЕ стал меньше критики.

# параметры топлива ====================
# по умолчанию заданы параметры для карамельного топлива 65% KNO₃ 35% C₁₂H₂₂O₁₁ (сахароза)
a = 8.2635  # параметр, задающий скорость горения
n = 0.319 # параметр, задающий скорость горения
# скорость горения определяется как u = a * p^n, где u - скорость (мм/с), p - давление (МПа)
# значения a и n для разных топлив см. здесь: http://kia-soft.narod.ru/interests/rockets/theory/burnlow/burnlow.htm
# оттуда взята и эта формула скорости горения 
density = 1.88 # плотность топлива, г/см^3
mol = 42. # молярная масса. Масса в граммах газа И ДЫМА на 1 моль ГАЗА.
t = 1450. + 273.15 # температура горения топлива. Температура нужна в °K, поэтому к температуре в °C прибавляется 273,15

I = 5.5 # среднее количество степеней свободы молекулы газа из выхлопа движка
# для отдельных атомов I = 3, для 2-атомных молекул - 5, для 3-атомных и с большим количеством атомов - 6.
# только если связи в молекуле жёсткие (как определить - не знаю). Усреднять нужно с учётом количества молекул.
smoke = 0.439 # массовая доля дыма в выхлопе
smoke_c = 1489. # теплоёмкость дыма, Дж/(кг*K)
# I, smoke и smoke_c нужны для расчёта показателя адиабаты k. Если k известен, просто укажите его. Тогда I, smoke и smoke_c не используются.
k = ... # коэффициент адиабаты  выхлопа

# параметры состава трассера ===========
# по умолчанию заданы параметры для карамельного топлива 65% KNO₃ 35% C₁₂H₂₂O₁₁
a_t = 1.2635
n_t = 0.319
density_t = 1.88 # плотность состава трассера, г/см^3
# отличия продуктов горения трассера и топлива не учитываются, т.к. доля продуктов сгорания трассера во всём выхлопе обычно мала
# после сгорания топлива скорость горения трассера расчитывается для атмосферного давления, что может быть некорректно, если трассер горит быстро
# поэтому используйте трассер из медленного состава или используйте другую программу для более правильного расчёта горения трассера

# поправки =============================
speed_k = 1. # отношение реальной скорости горения к теоретической
nozzle_friction = 1. # отношение реальной скорости выхода газов из сопла к теоретической. Сильно влияет на давление.
thrust_loss = 1. # отношение реальной тяги к теоретической

# другие параметры =====================
dots = 40. # точность симуляции, точек/мм
timestep = 1. / 400. # точность симуляции, длина временного шага, с
dynamic_nozzle_out = True # может ли поток газов "оторваться" от стенки сопла
out_pressure = 0.9 # давление, при котором поток отрывается от сопла, атм
# если dynamic_nozzle_out = True, значит, при слишком большом выходном диаметре сопла выхлопные газы оторвутся от сопла
# и сформируют струю раньше конца сопла, в момент когда их давление станет равным out_pressure
# тогда расчёт ведётся так, как будто выходной диаметр сопла равен тому диаметру в котором давление равно out_pressure
# если dynamic_nozzle_out = False, значит, газы продолжат идти по соплу и расширяться даже если их давление меньше атмосферного
# тогда тяга может стать даже отрицательной (из-за того что такая модель не подходит для некоторых случаев)
# как оно есть на самом деле - не знаю точно. Предположительно, если угол расширения сопла мал, а превышение оптимального выходного диаметра незначительно, dynamic_nozzle_out ближе к False
# если же выходной диаметр сопла меньше оптимального (т.е. давление у выхода сопла больше атмосферного и больше, чем out_pressure), значение dynamic_nozzle_out ни на что не влияет.

# параметры ракеты =====================
# здесь можно рассчитать высоту полёта ракеты, указав трение и массу
cf = 0.3 # коэффициент сопротивления формы ракеты (см. https://ru.wikipedia.org/wiki/Коэффициент_сопротивления_формы) (значение взято "с потолка" и может быть неправдоподобным)
air_density = 1.2 # плотость воздуха (нужна для расчёта лобового сопротивления), кг/м^3
rocket_volume = 400. # объём ракеты, см^3
friction = cf * air_density * (rocket_volume * 1e-6) ** .6666 * .5 # коэффициент, который нужно умножить на квадрат скорости (в м/с), чтобы получить силу трения, см. https://ru.wikipedia.org/wiki/Лобовое_сопротивление
# можете указать friction и тогда значения cf, air_density и rocket_volume не будут использоваться
mass = 1000. # масса ракеты, г

In [ ]:
# другой вариант параметров
# запустите эту ячейку кода после основной ячейки чтобы перезаписать только эти параметры

# параметры топлива ====================
# параметры для карамельного топлива 65% KNO₃ 35% C₆H₁₄O₆ (сорбит)
# эти параметры полностью взяты из Rocki-motors
a = 5.132  # параметр, задающий скорость горения
n = 0.322 # параметр, задающий скорость горения
density = 1.84 # плотность топлива, г/см^3
mol = 39.9 # молярная масса. Масса в граммах газа И ДЫМА на 1 моль ГАЗА.
t = 1600 # температура горения топлива в °K. Также в Roki-motors указана "практическая температура сгорания" в 1520 °K. Если нужно, замените.
k = 1.136 # показатель адиабаты выхлопа

In [ ]:
# simulation
# симуляция
# основной код вынесен в файл main.py, чтобы не занимать место на экране. https://notabug.org/busyBeaver/Rocket_motor_model/src/main/main.py
if not 'main' in globals():
    try:
        with open('main.py', 'r') as f: code = f.read()
    except FileNotFoundError:
        import requests; code = requests.get('https://notabug.org/busyBeaver/Rocket_motor_model/raw/main/main.py').text
    exec(code)
    main = init()
main(**globals())

## Ссылки

[Репозиторий с программой на NotABug](https://notabug.org/busyBeaver/Rocket_motor_model)

[Репозиторий с программой на GitHub](https://github.com/busyBeaver1/Rocket_motor_model)

[Закон горения](http://kia-soft.narod.ru/interests/rockets/theory/burnlow/burnlow.htm)

[Сайт автора статьи о законе горения и Rocki-motors](http://kia-soft.narod.ru/)

[Описание использованной здесь модель сопла - статья Ричарда Накки](https://www.nakka-rocketry.net/th_nozz.html)

[Видео про теорию ракетных двигателей и эту программу](https://www.youtube.com/watch?v=9XEeUQalxV4)

[Видео (не моё) про вторичный воспламеняющий состав](https://youtu.be/mQn43RASYNM)

[Вики про лобовое сопротивление воздуха](https://ru.wikipedia.org/wiki/Лобовое_сопротивление)

По всем вопросам и багам пишите на почту toegorbezrukov@yandex.ru.

#### Про использование программы | About copyright

Использование программы в исходном виде разрешается для любых целей. Повторное использование кода программы разрешается со ссылкой на оригинал. Условия могут уточняться.

The use of the program in its original form is allowed for any purpose. Reuse of the program code is allowed with link to the original. This conditions may be updated.